In [3]:
import os

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import BatchNormalization, Activation, LeakyReLU, UpSampling2D, Conv2D
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout
from tensorflow.keras.models import Sequential, Model

if not os.path.exists("./gan_images"):
    os.makedirs("./gan_images")


In [1]:

np.random.seed(3)
tf.random.set_seed(3)

#생성자 모델
def createGenerator():
    generator = Sequential()
    generator.add(Dense(128 * 7 * 7, input_dim=100, activation=LeakyReLU(0.2))) # -값이 제대로 학습되지 못하는 것을 막는다. 
    generator.add(BatchNormalization())                                         # 값들이 평균은 0 분산은 1이 되도록 값을 제 배열 시킨다. 
    generator.add(Reshape((7, 7, 128)))
    generator.add(UpSampling2D())                                               # UpSampling2D 함수는, 2차원 데이터를 두번씩 반복해서 해상도를 두 배 늘리는 함수다. 가로와 세로에 따라서 반복하는 횟수를 지정할 수도 있는데 기본 값은 두 배 씩 늘리는 것이다.  
    generator.add(Conv2D(64, kernel_size=5, padding='same'))
    generator.add(BatchNormalization())
    generator.add(Activation(LeakyReLU(0.2)))
    generator.add(UpSampling2D())
    generator.add(Conv2D(1, kernel_size=5, padding='same', activation='tanh'))

    return generator

#판별자 모델
def createDiscriminator():
    discriminator = Sequential()
    discriminator.add(Conv2D(64, kernel_size=5, strides=2, input_shape=(28, 28, 1), padding="same"))
    discriminator.add(Activation(LeakyReLU(0.2)))
    discriminator.add(Dropout(0.3))
    discriminator.add(Conv2D(128, kernel_size=5, strides=2, padding="same"))
    discriminator.add(Activation(LeakyReLU(0.2)))
    discriminator.add(Dropout(0.3))
    discriminator.add(Flatten())
    discriminator.add(Dense(1, activation='sigmoid'))

    return discriminator

# def createGan():


discriminator = createDiscriminator()
discriminator.compile(loss='binary_crossentropy', optimizer='adam')
discriminator.trainable = False

generator = createGenerator()
#생성자와 판별자 모델을 연결시키는 gan 모델
ginput = Input(shape=(100,))
dis_output = discriminator(generator(ginput))
gan = Model(ginput, dis_output)
gan.compile(loss='binary_crossentropy', optimizer='adam')
gan.summary()

#신경망을 실행시키는 함수
def gan_train(epoch, batch_size, saving_interval):
  # MNIST 데이터 불러오기
  (X_train, _), (_, _) = mnist.load_data()  # 앞서 불러온 적 있는 MNIST를 다시 이용합니다. 단, 테스트과정은 필요없고 이미지만 사용할 것이기 때문에 X_train만 불러왔습니다.
  X_train = X_train.reshape(X_train.shape[0], 28, 28, 1).astype('float32')
  X_train = (X_train - 127.5) / 127.5  # 픽셀값은 0에서 255사이의 값입니다. 이전에 255로 나누어 줄때는 이를 0~1사이의 값으로 바꾸었던 것인데, 여기서는 127.5를 빼준 뒤 127.5로 나누어 줌으로 인해 -1에서 1사이의 값으로 바뀌게 됩니다.
  #X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

  true = np.ones((batch_size, 1))
  fake = np.zeros((batch_size, 1))

  for i in range(epoch):
      # 실제 데이터를 판별자에 입력하는 부분입니다.
      idx = np.random.randint(0, X_train.shape[0], batch_size)
      imgs = X_train[idx]
      d_loss_real = discriminator.train_on_batch(imgs, true)

      #가상 이미지를 판별자에 입력하는 부분입니다.
      noise = np.random.normal(0, 1, (batch_size, 100))
      gen_imgs = generator.predict(noise)
      d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)

      #판별자와 생성자의 오차를 계산합니다.
      d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
      g_loss = gan.train_on_batch(noise, true)

      print('epoch:%d' % i, ' d_loss:%.4f' % d_loss, ' g_loss:%.4f' % g_loss)

    # 이부분은 중간 과정을 이미지로 저장해 주는 부분입니다. 본 장의 주요 내용과 관련이 없어
    # 소스코드만 첨부합니다. 만들어진 이미지들은 gan_images 폴더에 저장됩니다.
      if i % saving_interval == 0:
          #r, c = 5, 5
          noise = np.random.normal(0, 1, (25, 100))
          gen_imgs = generator.predict(noise)

          # Rescale images 0 - 1
          gen_imgs = 0.5 * gen_imgs + 0.5

          fig, axs = plt.subplots(5, 5)
          count = 0
          for j in range(5):
              for k in range(5):
                  axs[j, k].imshow(gen_imgs[count, :, :, 0], cmap='gray')
                  axs[j, k].axis('off')
                  count += 1
          fig.savefig("gan_images/gan_mnist_%d.png" % i)

gan_train(4001, 32, 200)  #4000번 반복되고(+1을 해 주는 것에 주의), 배치 사이즈는 32,  200번 마다 결과가 저장되게 하였습니다.


loss:0.4024  g_loss:1.3222
epoch:73  d_loss:0.3386  g_loss:1.6580
epoch:74  d_loss:0.3693  g_loss:1.5531
epoch:75  d_loss:0.3771  g_loss:1.6133
epoch:76  d_loss:0.3793  g_loss:1.6082
epoch:77  d_loss:0.3682  g_loss:1.5375
epoch:78  d_loss:0.3784  g_loss:1.5319
epoch:79  d_loss:0.3937  g_loss:1.5528
epoch:80  d_loss:0.3481  g_loss:2.1415
epoch:81  d_loss:0.5119  g_loss:1.7801
epoch:82  d_loss:0.4032  g_loss:1.8850
epoch:83  d_loss:0.4502  g_loss:1.4909
epoch:84  d_loss:0.5745  g_loss:1.8139
epoch:85  d_loss:0.4681  g_loss:2.2931
epoch:86  d_loss:0.4985  g_loss:2.5427
epoch:87  d_loss:0.5349  g_loss:2.7520
epoch:88  d_loss:0.6341  g_loss:1.5436
epoch:89  d_loss:0.5908  g_loss:1.5701
epoch:90  d_loss:0.3744  g_loss:2.2424
epoch:91  d_loss:0.6131  g_loss:2.2821
epoch:92  d_loss:0.4968  g_loss:2.2830
epoch:93  d_loss:0.6531  g_loss:1.5912
epoch:94  d_loss:0.6852  g_loss:1.3946
epoch:95  d_loss:0.6481  g_loss:1.4877
epoch:96  d_loss:0.6213  g_loss:1.7969
epoch:97  d_loss:0.7328  g_loss:1.613

KeyboardInterrupt: 

In [7]:
# Upsampling 각각을 업 샘플링 하여 데이터를 변화 시킨다. 
input_shape = (2, 2, 1, 3)
x = np.arange(np.prod(input_shape)).reshape(input_shape)
print(x)
y = tf.keras.layers.UpSampling2D(size=(1, 2))(x)
print(y)



[[[[ 0  1  2]]

  [[ 3  4  5]]]


 [[[ 6  7  8]]

  [[ 9 10 11]]]]
tf.Tensor(
[[[[ 0  1  2]
   [ 0  1  2]]

  [[ 3  4  5]
   [ 3  4  5]]]


 [[[ 6  7  8]
   [ 6  7  8]]

  [[ 9 10 11]
   [ 9 10 11]]]], shape=(2, 2, 2, 3), dtype=int32)
